In [2]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

def encontrar_data_anterior_mais_proxima(lista_datas_str, data_referencia):
  datas = []
  for data_str in lista_datas_str:
    try:
      datas.append(datetime.strptime(data_str,  "%Y-%m-%d %H:%M:%S.%f"))
    except ValueError:
      print(f"Data inválida: {data_str}")
  datas_anteriores = [data for data in datas if data <= data_referencia]

  if datas_anteriores:
    return max(datas_anteriores).strftime( "%Y-%m-%d %H:%M:%S.%f")[:-3]
  else:
    return None

def encontrar_data_posterior_mais_proxima(lista_datas_str, data_referencia):
  datas = []
  for data_str in lista_datas_str:
    try:
      datas.append(datetime.strptime(data_str, "%Y-%m-%d %H:%M:%S.%f"))
    except ValueError:
      print(f"Data inválida: {data_str}")
  datas_posteriores = [data for data in datas if data >= data_referencia]

  if datas_posteriores:
    return min(datas_posteriores).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
  else:
    return None

dia = mes = 0
listaQtdAlocacoes = []
colunas = ['RN','CLIENTE', 'CD_ALOCACAO', 'DATA_HORA_VAZAO', 'TURNO', 'DATA_SAIDA_CAMINHAO','DATA_CHEGADA_CAMINHAO',
           'DESCRICAO_TIPO_CARGA','PESO_PADRAO','DISTANCIA','VAZAO_FRENTE','CLASSIFICACAO']
contadorParadas = 0
datasetAjustado = pd.DataFrame(columns=colunas)
for i in range(4, 13):
  if i == 13:
    break
  url = f"https://raw.githubusercontent.com/RafaelGirardin/ModeloPredicaoFazenda/refs/heads/master/USL/{str(i)}.csv"
  colunas = ['RN','CLIENTE', 'CD_ALOCACAO', 'DATA_HORA_VAZAO', 'TURNO', 'DATA_SAIDA_CAMINHAO','DATA_CHEGADA_CAMINHAO',
           'DESCRICAO_TIPO_CARGA','PESO_PADRAO','DISTANCIA','VAZAO_FRENTE', 'SALDO_VAZAO', 'EXCESSO_CAMINHAO', 'CD_FAZENDA_LIBERADA', 'DESCRICAO_FAZENDA', 'ALOCACAO_COM_PARADA', 'CLASSIFICACAO']
  datasetAuxiliar = pd.read_csv(url, names=colunas, skiprows=1, delimiter=',')
  datasetAuxiliar['DATA_SAIDA_CAMINHAO'] = pd.to_datetime(datasetAuxiliar['DATA_SAIDA_CAMINHAO'])
  datasetAuxiliar.head(100)
  for j in range(1,32):

    if (i == 4 or i == 6 or i == 9 or i == 11) and (j == 31):
      break
    if (i == 2 and j == 30):
      break
    if (i == 2 and j < 21):
      continue
    dia = str(j)
    if j < 10:
      dia = "0" + str(j)
      mes = str(i)
    if i < 10:
      mes = "0" + str(i)

    data_especifica = pd.to_datetime(f'2024-{mes}-{dia}')
    datasetOriginal = datasetAuxiliar[datasetAuxiliar['DATA_SAIDA_CAMINHAO'].dt.date == data_especifica.date()]
    datasetOriginal['DATA_SAIDA_CAMINHAO'] = datasetOriginal['DATA_SAIDA_CAMINHAO'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')
    datasetOriginal['DATA_SAIDA_CAMINHAO']= datasetOriginal['DATA_SAIDA_CAMINHAO'].apply(lambda x: x[:-3])
    datasetOriginal['CLASSIFICACAO'] = "Erro"
    datasetOriginal['TEMPO_INFERIOR'] = datasetOriginal['MARGEM_INFERIOR'] = datasetOriginal['MARGEM_SUPERIOR'] = 0
    listaQtdAlocacoes.append(len(datasetOriginal['CD_ALOCACAO']))
    if len(datasetOriginal['CD_ALOCACAO']) == 0:
      continue
    percVazio = (len(datasetOriginal.query('VAZAO_FRENTE.isnull()', engine='python')) + len(datasetOriginal.query('VAZAO_FRENTE == 0', engine='python')))/len(datasetOriginal['VAZAO_FRENTE'])
    if percVazio >= 0.175:
      continue

    alocacoesDatetime = pd.to_datetime(datasetOriginal['DATA_SAIDA_CAMINHAO'])
    alocacoesManha = pd.to_datetime(f'2024-{mes}-{dia} 00:00:00')
    alocacoesTarde = pd.to_datetime(f'2024-{mes}-{dia} 06:00:00')
    alocacoesTardezinha = pd.to_datetime(f'2024-{mes}-{dia} 12:00:00')
    alocacoesNoite = pd.to_datetime(f'2024-{mes}-{dia} 18:00:00')
    qtdAlocacoesManha = alocacoesDatetime[(alocacoesDatetime > alocacoesManha) & (alocacoesDatetime < alocacoesTarde)]
    qtdAlocacoesTarde = alocacoesDatetime[(alocacoesDatetime > alocacoesTarde) & (alocacoesDatetime < alocacoesTardezinha)]
    qtdAlocacoesTardezinha = alocacoesDatetime[(alocacoesDatetime > alocacoesTardezinha) & (alocacoesDatetime < alocacoesNoite)]
    qtdAlocacoesNoite = alocacoesDatetime[(alocacoesDatetime >= alocacoesNoite)]
    if (qtdAlocacoesNoite.empty or qtdAlocacoesTarde.empty or qtdAlocacoesManha.empty or qtdAlocacoesTardezinha.empty):
      continue

    print(dia + "-" + mes)
    D1 = pd.to_datetime(datasetOriginal['DATA_CHEGADA_CAMINHAO']).quantile(0.025)
    D9 = pd.to_datetime(datasetOriginal['DATA_CHEGADA_CAMINHAO']).quantile(0.975)
    datasetOriginal['DATA_CHEGADA_CAMINHAO'] = pd.to_datetime(datasetOriginal['DATA_CHEGADA_CAMINHAO'])
    datasetOriginal = datasetOriginal[datasetOriginal['DATA_CHEGADA_CAMINHAO'] >= D1]
    datasetOriginal = datasetOriginal[datasetOriginal['DATA_CHEGADA_CAMINHAO'] <= D9]
    datasetOriginal['DATA_CHEGADA_CAMINHAO'] = datasetOriginal['DATA_CHEGADA_CAMINHAO'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')
    datasetOriginal = datasetOriginal.reset_index(drop=True)

    datasetDoDiaComVazoesPreenchidas = datasetOriginal.query('(not VAZAO_FRENTE.isnull()) and VAZAO_FRENTE > 0', engine='python')
    for vazao in range(0, len(datasetOriginal['VAZAO_FRENTE'])):
      if np.isnan(datasetOriginal['VAZAO_FRENTE'][vazao]) or datasetOriginal['VAZAO_FRENTE'][vazao] == 0:
        distancia = datasetOriginal['DISTANCIA'][vazao]
        datasetDoDiaComVazoesPreenchidasParaUmaFazenda = datasetDoDiaComVazoesPreenchidas.loc[(datasetOriginal['DISTANCIA'] == distancia)]
        datetimeDataVazaoNula = datetime.strptime(datasetOriginal['DATA_HORA_VAZAO'][vazao],"%Y-%m-%d %H:%M:%S.%f")
        stringDataVazaoNaoNulaAnteriorMaisProxima = encontrar_data_anterior_mais_proxima(datasetDoDiaComVazoesPreenchidasParaUmaFazenda['DATA_HORA_VAZAO'], datetimeDataVazaoNula)
        stringDataVazaoNaoNulaPosteriorMaisProxima = encontrar_data_posterior_mais_proxima(datasetDoDiaComVazoesPreenchidasParaUmaFazenda['DATA_HORA_VAZAO'], datetimeDataVazaoNula)
        if stringDataVazaoNaoNulaAnteriorMaisProxima is None and stringDataVazaoNaoNulaPosteriorMaisProxima is None:
          if len(datasetDoDiaComVazoesPreenchidas) == 0:
            break
          somaVazoes = 0
          for vazaoNaoNula in range(0, len(datasetDoDiaComVazoesPreenchidas)):
            somaVazoes += datasetDoDiaComVazoesPreenchidas.iloc[vazaoNaoNula]['VAZAO_FRENTE']
          mediaVazoesDiferentesFazendas = somaVazoes/len(datasetDoDiaComVazoesPreenchidas)
          datasetOriginal['VAZAO_FRENTE'][vazao] = mediaVazoesDiferentesFazendas
          continue
        elif stringDataVazaoNaoNulaAnteriorMaisProxima is None:
          stringDataVazaoNaoNulaAnteriorMaisProxima = stringDataVazaoNaoNulaPosteriorMaisProxima
        elif stringDataVazaoNaoNulaPosteriorMaisProxima is None:
          stringDataVazaoNaoNulaPosteriorMaisProxima = stringDataVazaoNaoNulaAnteriorMaisProxima
        linhasComVazoesNaoNulasAnterioresMaisProximas = datasetOriginal.loc[datasetOriginal['DATA_HORA_VAZAO'] == stringDataVazaoNaoNulaAnteriorMaisProxima]
        linhasComVazoesNaoNulasPosterioresMaisProximas = datasetOriginal.loc[datasetOriginal['DATA_HORA_VAZAO'] == stringDataVazaoNaoNulaPosteriorMaisProxima]
        vazaoNaoNulaAnteriorMaisProxima = list(linhasComVazoesNaoNulasAnterioresMaisProximas['VAZAO_FRENTE'])[0]
        vazaoNaoNulaPosteriorMaisProxima = list(linhasComVazoesNaoNulasPosterioresMaisProximas['VAZAO_FRENTE'])[0]
        mediaVazoesNaoNulasMaisProximas = (vazaoNaoNulaAnteriorMaisProxima + vazaoNaoNulaPosteriorMaisProxima)/2
        datasetOriginal['VAZAO_FRENTE'][vazao] = mediaVazoesNaoNulasMaisProximas
    for alocacao in range(1, len(datasetOriginal['CD_ALOCACAO'])):
      if pd.isna(datasetOriginal['DATA_CHEGADA_CAMINHAO'][alocacao]) or datasetOriginal['ALOCACAO_COM_PARADA'][alocacao] == 'SIM':
        if (not pd.isna(datasetOriginal['DATA_CHEGADA_CAMINHAO'][alocacao])) and datasetOriginal['ALOCACAO_COM_PARADA'][alocacao] == 'SIM':
            contadorParadas += 1
        datasChegadasPreenchidas = datasetOriginal.loc[datasetOriginal['DATA_CHEGADA_CAMINHAO'].notna()]
        datasChegadasPreenchidasParaUmaFazenda = datasChegadasPreenchidas.loc[datasChegadasPreenchidas['DISTANCIA'] == datasetOriginal['DISTANCIA'][alocacao]]
        datasChegadasPreenchidasParaUmaFazendaEPeso = datasChegadasPreenchidasParaUmaFazenda.loc[datasChegadasPreenchidasParaUmaFazenda['PESO_PADRAO'] == datasetOriginal['PESO_PADRAO'][alocacao]]
        somaTempoDasViagens = mediaTempoDasViagens = 0
        if len(datasChegadasPreenchidasParaUmaFazendaEPeso) != 0:
          for data in range(0, len(datasChegadasPreenchidasParaUmaFazendaEPeso)):
            somaTempoDasViagens += (datetime.strptime(datasChegadasPreenchidasParaUmaFazendaEPeso.iloc[data]['DATA_CHEGADA_CAMINHAO'],"%Y-%m-%d %H:%M:%S.%f") - datetime.strptime(datasChegadasPreenchidasParaUmaFazendaEPeso.iloc[data]['DATA_SAIDA_CAMINHAO'],"%Y-%m-%d %H:%M:%S.%f")).total_seconds()/60
            mediaTempoDasViagens = somaTempoDasViagens/len(datasChegadasPreenchidasParaUmaFazendaEPeso)
        elif len(datasChegadasPreenchidasParaUmaFazenda) != 0:
          for data in range(0, len(datasChegadasPreenchidasParaUmaFazenda)):
            somaTempoDasViagens += (datetime.strptime(datasChegadasPreenchidasParaUmaFazenda.iloc[data]['DATA_CHEGADA_CAMINHAO'],"%Y-%m-%d %H:%M:%S.%f") - datetime.strptime(datasChegadasPreenchidasParaUmaFazenda.iloc[data]['DATA_SAIDA_CAMINHAO'],"%Y-%m-%d %H:%M:%S.%f")).total_seconds()/60
            mediaTempoDasViagens = somaTempoDasViagens/len(datasChegadasPreenchidasParaUmaFazenda)
        else:
          for data in range(0, len(datasChegadasPreenchidas)):
            somaTempoDasViagens += (datetime.strptime(datasChegadasPreenchidas.iloc[data]['DATA_CHEGADA_CAMINHAO'],"%Y-%m-%d %H:%M:%S.%f") - datetime.strptime(datasChegadasPreenchidas.iloc[data]['DATA_SAIDA_CAMINHAO'],"%Y-%m-%d %H:%M:%S.%f")).total_seconds()/60
            mediaTempoDasViagens = somaTempoDasViagens/len(datasChegadasPreenchidas)
        tempoPrevistoViagem = timedelta(minutes=mediaTempoDasViagens)
        datasetOriginal['DATA_CHEGADA_CAMINHAO'][alocacao] = (datetime.strptime(datasetOriginal['DATA_SAIDA_CAMINHAO'][alocacao],"%Y-%m-%d %H:%M:%S.%f") + tempoPrevistoViagem).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    datasetOriginal['DATA_CHEGADA_CAMINHAO'] = pd.to_datetime(datasetOriginal['DATA_CHEGADA_CAMINHAO'])
    datasetOriginal = datasetOriginal.sort_values(by='DATA_CHEGADA_CAMINHAO', ascending=True).reset_index(drop=True)
    datasetOriginal['DATA_CHEGADA_CAMINHAO'] = datasetOriginal['DATA_CHEGADA_CAMINHAO'].dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    amplitudeDeHorariosDaAlocacao = max([datetime.strptime(data, "%Y-%m-%d %H:%M:%S.%f") for data in datasetOriginal['DATA_SAIDA_CAMINHAO']]) - min([datetime.strptime(data, "%Y-%m-%d %H:%M:%S.%f") for data in datasetOriginal['DATA_SAIDA_CAMINHAO']])
    amplitudeEmMinutosDeHorariosDaAlocacao = float((amplitudeDeHorariosDaAlocacao.total_seconds()) / 60)
    margemSuperior = amplitudeEmMinutosDeHorariosDaAlocacao/(len(datasetOriginal['DATA_SAIDA_CAMINHAO']) - 1) + (amplitudeEmMinutosDeHorariosDaAlocacao)/(len(datasetOriginal['DATA_SAIDA_CAMINHAO']) - 1)*0.4
    margemInferior = amplitudeEmMinutosDeHorariosDaAlocacao/(len(datasetOriginal['DATA_SAIDA_CAMINHAO']) - 1) - (amplitudeEmMinutosDeHorariosDaAlocacao)/(len(datasetOriginal['DATA_SAIDA_CAMINHAO']) - 1)*0.4
    for m in range(1,len(datasetOriginal['DATA_CHEGADA_CAMINHAO'])):
      tempoInferior = pd.to_datetime(datasetOriginal['DATA_CHEGADA_CAMINHAO'][m], format='%Y-%m-%d %H:%M:%S.%f') - pd.to_datetime(datasetOriginal['DATA_CHEGADA_CAMINHAO'][m-1], format='%Y-%m-%d %H:%M:%S.%f')
      if (tempoInferior).seconds/60 >=  margemInferior and (tempoInferior).seconds/60 <= margemSuperior:
        datasetOriginal['CLASSIFICACAO'][m] = "A"
      elif (tempoInferior).seconds/60 >=  margemInferior/1.25 and (tempoInferior).seconds/60 <= margemSuperior*1.25:
        datasetOriginal['CLASSIFICACAO'][m] = "B"
      elif (tempoInferior).seconds/60 >=  margemInferior/2.0 and (tempoInferior).seconds/60 <= margemSuperior*2.0:
        datasetOriginal['CLASSIFICACAO'][m] = "C"
      elif (tempoInferior).seconds/60 >=  margemInferior/4.0 and (tempoInferior).seconds/60 <= margemSuperior*4.0:
        datasetOriginal['CLASSIFICACAO'][m] = "D"
      else:
        datasetOriginal['CLASSIFICACAO'][m] = "E"
      datasetOriginal['TEMPO_INFERIOR'][m] = tempoInferior.seconds/60
      datasetOriginal['MARGEM_INFERIOR'][m] = margemInferior
      datasetOriginal['MARGEM_SUPERIOR'][m] = margemSuperior
    datasetAjustado = pd.concat([datasetAjustado, datasetOriginal],ignore_index=True)
datasetAjustado = datasetAjustado[datasetAjustado['CLASSIFICACAO'] != 'Erro']
print(datasetAjustado.shape)
datasetAjustado = datasetAjustado.drop(['RN'], axis=1)
datasetAjustado = datasetAjustado.drop(['CLIENTE'], axis=1)
print(contadorParadas)
datasetAjustado.head(35000)

08-05
09-05
10-05
11-05
12-05
13-05
14-05
15-05
16-05
17-05
18-05


KeyboardInterrupt: 

In [7]:
datasetAuxiliar = datasetAjustado
datasetAuxiliar = datasetAuxiliar.drop(['CD_ALOCACAO'], axis=1)
datasetAuxiliar = datasetAuxiliar.drop(['DATA_HORA_VAZAO'], axis=1)
datasetAuxiliar = datasetAuxiliar.drop(['DESCRICAO_TIPO_CARGA'], axis=1)
datasetAuxiliar = datasetAuxiliar.drop(['CD_FAZENDA_LIBERADA'], axis=1)
datasetAuxiliar = datasetAuxiliar.drop(['DESCRICAO_FAZENDA'], axis=1)
datasetAuxiliar.head(12000)

,TURNO,DATA_SAIDA_CAMINHAO,DATA_CHEGADA_CAMINHAO,PESO_PADRAO,DISTANCIA,VAZAO_FRENTE,CLASSIFICACAO,SALDO_VAZAO,EXCESSO_CAMINHAO,ALOCACAO_COM_PARADA,TEMPO_INFERIOR,MARGEM_INFERIOR,MARGEM_SUPERIOR
1,NOITE,2024-02-21 05:55:34.000,2024-02-21 11:16:05.705000,60,59.05,34.19,C,0.000000,S,SIM,22.133333,6.653333,15.524444
2,MANHA,2024-02-21 10:25:50.000,2024-02-21 11:43:52.000000,60,59.05,55.91,C,0.000000,S,NAO,27.766667,6.653333,15.524444
3,NOITE,2024-02-21 05:08:52.000,2024-02-21 11:49:06.000000,60,59.05,73.51,C,0.000000,N,NAO,5.233333,6.653333,15.524444
4,NOITE,2024-02-21 05:24:23.000,2024-02-21 12:30:25.000000,60,59.05,77.36,D,0.000000,S,NAO,41.316667,6.653333,15.524444
5,MANHA,2024-02-21 09:22:41.000,2024-02-21 13:07:14.000000,60,57.71,205.95,D,0.000000,S,NAO,36.816667,6.653333,15.524444
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,MANHA,2024-11-28 17:41:46.000,2024-11-28 22:10:49.000000,60,70.58,555.67,A,58.326000,S,NAO,8.316667,5.489915,12.809802
6142,MANHA,2024-11-28 17:53:29.000,2024-11-28 22:15:16.000000,60,70.58,538.82,B,58.712000,S,NAO,4.450000,5.489915,12.809802
6143,MANHA,2024-11-28 17:53:54.000,2024-11-28 22:21:10.065000,60,70.58,538.82,A,100.284444,S,SIM,5.900000,5.489915,12.809802
6144,MANHA,2024-11-28 17:55:30.000,2024-11-28 22:21:45.121000,60,70.58,542.64,E,58.326000,S,SIM,0.583333,5.489915,12.809802


In [11]:
preDatasetTreinamento = datasetAuxiliar
preDatasetTreinamento = preDatasetTreinamento.drop(['DATA_SAIDA_CAMINHAO'], axis=1)
preDatasetTreinamento = preDatasetTreinamento.drop(['DATA_CHEGADA_CAMINHAO'], axis=1)
preDatasetTreinamento = preDatasetTreinamento.drop(['TEMPO_INFERIOR'], axis=1)
preDatasetTreinamento = preDatasetTreinamento.drop(['MARGEM_INFERIOR'], axis=1)
preDatasetTreinamento = preDatasetTreinamento.drop(['MARGEM_SUPERIOR'], axis=1)
preDatasetTreinamento.head(12000)

,TURNO,PESO_PADRAO,DISTANCIA,VAZAO_FRENTE,CLASSIFICACAO,SALDO_VAZAO,EXCESSO_CAMINHAO,ALOCACAO_COM_PARADA
1,NOITE,60,59.05,34.19,C,0.000000,S,SIM
2,MANHA,60,59.05,55.91,C,0.000000,S,NAO
3,NOITE,60,59.05,73.51,C,0.000000,N,NAO
4,NOITE,60,59.05,77.36,D,0.000000,S,NAO
5,MANHA,60,57.71,205.95,D,0.000000,S,NAO
...,...,...,...,...,...,...,...,...
6141,MANHA,60,70.58,555.67,A,58.326000,S,NAO
6142,MANHA,60,70.58,538.82,B,58.712000,S,NAO
6143,MANHA,60,70.58,538.82,A,100.284444,S,SIM
6144,MANHA,60,70.58,542.64,E,58.326000,S,SIM
